<a href="https://colab.research.google.com/github/Berkay86/DeepPerson/blob/main/POLARITY%20SENTIMENT%20ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Kütüphanelerimizi içeriye alalım.
#Veri İşlemleri
!pip3 install snscrape
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string


In [3]:
import re 
import textblob
from textblob import TextBlob

In [4]:
from wordcloud.wordcloud import WordCloud,STOPWORDS
!pip install emot
from emot.emo_unicode import UNICODE_EMOJI
lemmatizer=WordNetLemmatizer()
from wordcloud import ImageColorGenerator
from PIL import Image
import warnings
%matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using matplotlib backend: agg


In [5]:
query="(worldcup ))"
tweets=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
  if i>2000:
    break
  else:
        tweets.append([tweet.date,tweet.id,tweet.url,tweet.user.username,tweet.sourceLabel,tweet.user.location,tweet.content,tweet.likeCount,tweet.retweetCount])
df=pd.DataFrame(tweets,columns=["Date","ID","url","username","source","location","tweet","num_of_likes", "num_of_retweet"])
df.to_csv("sentiment.csv",encoding='utf-8')
df

,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,2022-12-12 11:27:53+00:00,1602263969460371457,https://twitter.com/vvvrba/status/160226396946...,vvvrba,Twitter for iPhone,,"@ESPNFC it’s just meant to be for #Modric, his...",0,0
1,2022-12-12 11:27:53+00:00,1602263969074380803,https://twitter.com/ribeiro_v3/status/16022639...,ribeiro_v3,Twitter Web App,,@Iuuk96 @PortugalTaIk How was he carrying this...,0,0
2,2022-12-12 11:27:53+00:00,1602263967405166595,https://twitter.com/NureenKhadr/status/1602263...,NureenKhadr,Twitter for iPhone,SFO - CAI - DXB,".@netflix / @NetflixMENA , I think you should ...",0,0
3,2022-12-12 11:27:52+00:00,1602263963277791232,https://twitter.com/ViktorAllen3/status/160226...,ViktorAllen3,Twitter Web App,,@FIFAWorldCup Tons of respect for Fernando Hie...,0,0
4,2022-12-12 11:27:50+00:00,1602263956025794562,https://twitter.com/KinKinetic/status/16022639...,KinKinetic,Twitter Web App,,Gm Gm fam.\n\nWojakQatar is on @Danksetio \nY...,0,0
...,...,...,...,...,...,...,...,...,...
1996,2022-12-12 10:53:45+00:00,1602255379605458944,https://twitter.com/Fawad57696644/status/16022...,Fawad57696644,Twitter for iPhone,,@FPL_JD Told you mbappe wouldn’t do fuck all. ...,1,0
1997,2022-12-12 10:53:45+00:00,1602255379211161601,https://twitter.com/soufhilal/status/160225537...,soufhilal,Twitter Web App,Casablanca,@UTDTrey The one who is finished is the one wh...,0,0
1998,2022-12-12 10:53:45+00:00,1602255378326118401,https://twitter.com/maliquexv/status/160225537...,maliquexv,Twitter for Android,,"@sportbible Even england have messi, maradona,...",0,0
1999,2022-12-12 10:53:44+00:00,1602255376631595009,https://twitter.com/WillyWartaal1/status/16022...,WillyWartaal1,Twitter Web App,,@Stephen_750 @paddypower @NeilRedFM According ...,0,0


In [6]:
nltk.download('stopwords')
turk_stop_words=list(stopwords.words("turkish"))
emoji=list(UNICODE_EMOJI.keys())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df["location"]=df["location"].fillna("Unknown")

In [8]:
def ProcessedTweets(text):
  text=text.lower()
  text=" ".join(re.sub("(@[A-Za-z0-9]+)|(^0-9A-Za-z \t]) |(\w+:\/\/\S+)"," ",text).split()) 
  text=re.sub(r'\@\w+|\#\w+|\d+', "", text)
  punct=str.maketrans("","",string.punctuation+string.digits)
  text=text.translate(punct)
  tokens=word_tokenize(text)
  filtered_words=[w for w in tokens if w not in turk_stop_words]
  filtered_words=[w for w in filtered_words if w not in emoji]
  lemmatizer=WordNetLemmatizer()
  lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
  text=" ".join(lemma_words)
  return text



In [9]:
tokenizer = nltk.data.load('nltk:tokenizers/punkt/turkish.pickle')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
df["Processed_Tweets"]=df["tweet"].apply(ProcessedTweets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
df["Processed_Tweets"]

0       it ’ s just meant to be for his grandpa died o...
1       how wa he carrying this world cup lol he got o...
2       i think you should know we are all hoping you ...
3       ton of respect for fernando hierro but bono th...
4               gm gm fam wojakqatar is on la la ha bi bi
                              ...                        
1996    jd told you mbappe wouldn ’ t do fuck all only...
1997    the one who is finished is the one who ha been...
1998    even england have messi maradonapeleronaldo cr...
1999    according to the official document he is not b...
2000               argentina will win this worldcup🇦🇷🇦🇷🇦🇷
Name: Processed_Tweets, Length: 2001, dtype: object

In [11]:
def polarity(tweet):
  return TextBlob(tweet).sentiment.polarity
def sentimenttextblob(polarity):
  if polarity<0:
    return "Negative"
  elif polarity>0:
    return "Positive"


In [12]:
df["Polarity"]=df["Processed_Tweets"].apply(polarity)
df["Sentiment"]=df["Polarity"].apply(sentimenttextblob)
sent=df["Sentiment"].value_counts()
sent


Positive    862
Negative    314
Name: Sentiment, dtype: int64